<h1 style="color:red;">Coursera Capstone project - Pt 1</h1>

In [24]:
import pandas as pd
import numpy as np

In [25]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Convert all the tables to dataframe from the site

In [215]:
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

### Selec table 0 which is our aim

In [232]:
df = table[0]

### Select only the rows which don't have 'not assigned' in Borough column

In [233]:
mask = df.Borough != 'Not assigned'
df = df[mask]

### Get list of neighboors for each postcode

In [234]:
grouped_df = df.groupby(by=['Postcode','Borough'])
post_nb = dict()
for postcode, group in grouped_df:
    post_nb[postcode[0]] = set(list(group.Neighbourhood))

### Create a new dataframe, with neighboorhood aggregated in a list

In [235]:
def get_Borough(row):
    row.Borough = df[df.Postcode == row.Postcode].Borough.to_list()[0]
    try:
        row.Neighbourhood.remove('Not assigned')
    except:
        pass
    row.Neighbourhood = ', '.join(row.Neighbourhood)

In [246]:
grouped_df = pd.DataFrame(columns=df.columns)
grouped_df.Postcode = post_nb.keys()
grouped_df.Neighbourhood = post_nb.values()
grouped_df.apply(get_Borough, axis=1).reset_index(name='Neighbourhood');

In [247]:
grouped_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"Morningside, West Hill, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Replace all 'Not assigned' neirborhoods for the value of borought of the respective row

In [249]:
def replace_func(row):
    if row.Neighbourhood == 'Not assigned':
        row.Neighbourhood = row.Borough      

In [253]:
grouped_df.apply(replace_func,axis=1)
grouped_df.reset_index(drop=True,inplace=True)
grouped_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"Morningside, West Hill, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [254]:
grouped_df.shape 

(103, 3)

<h1 style = "color:red;">END OF PART 1</h1>

<h1 style="color:green;"> GET GEOSPATIAL DATA - PART 2</h1>

In [225]:
! conda install geocoder --yes
import geocoder # import geocoder

Solving environment: done

# All requested packages already installed.



In [226]:
"""def get_lat_lng(row):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(row.Postcode))
        lat_lng_coords = g.latlng

    row['Lat'] = lat_lng_coords[0]
    row['Lng'] = lat_lng_coords[1]
grouped_df.apply(get_lat_lng,axis=1)
grouped_df
""";

In [227]:
geospatial_df = pd.read_csv('Geospatial_Coordinates.csv')
geospatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [263]:
def get_lat_lng(row):
    geo_row = geospatial_df[geospatial_df['Postal Code'] == row.Postcode]
    row.Latitude = geo_row.Latitude.to_list()[0]
    row.Longitude = geo_row.Longitude.to_list()[0]
grouped_df["Latitude"] = ""
grouped_df["Longitude"] = ""
grouped_df.apply(get_lat_lng,axis=1,result_type='expand')
grouped_df.head(11)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Morningside, West Hill, Guildwood",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"Ionview, East Birchmount Park, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Oakridge, Clairlea, Golden Mile",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside",43.7163,-79.2395
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.6927,-79.2648


<h1 style="color:green;">END OF PART 2 </h1>

<h1 style="color:blue;">Explore and cluster the neighborhoods in Toronto - Part 3</h1>

In [268]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

### Get only the Borough that contains Toronto

In [343]:
toronto_df = grouped_df[ grouped_df.Borough.str.contains('Toronto') ]

In [275]:
map_toronto = folium.Map(location=[43.4200, -79.2458], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [276]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [289]:
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.6764,-79.293
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6796,-79.3522
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.669,-79.3156
43,M4M,East Toronto,Studio District,43.6595,-79.3409
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888


#### Let's explore the first neighborhood in our dataframe

In [279]:
neighborhood_latitude = toronto_df.iloc[0,3] # neighborhood latitude value
neighborhood_longitude = toronto_df.iloc[0,4] # neighborhood longitude value

neighborhood_name = toronto_df.iloc[0,2] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues that are in East Toronto within a radius of 700 meters

In [282]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 700 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

#### Make get request

In [283]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e3b8095542890001b3fee8e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 41,
  'suggestedBounds': {'ne': {'lat': 43.6826574063, 'lng': -79.28433680028674},
   'sw': {'lat': 43.670057393699985, 'lng': -79.30172559971327}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'di

In [285]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now let's convert json we got from foursquare to pandas dataframe

In [287]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Beaches Bake Shop,Bakery,43.680363,-79.289692
2,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
3,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272
4,The Beech Tree,Gastropub,43.680493,-79.288846


#### How many venues were returned by foursquare?

In [288]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

41 venues were returned by Foursquare.


<h4> Explore Neighborhoods in Toronto</h4>
<h4>Let's create a function to repeat the same process to all the neighborhoods in Toronto</h4>

In [290]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [292]:
#Run the function above
toronto_venues = getNearbyVenues(toronto_df['Neighbourhood'], toronto_df['Latitude'], toronto_df['Longitude'])



The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East, Moore Park
Rathnelly, Forest Hill SE, Deer Park, Summerhill West, South Hill
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Union Station, Toronto Islands, Harbourfront East
Toronto Dominion Centre, Design Exchange
Victoria Hotel, Commerce Court
Roselawn
Forest Hill North, Forest Hill West
Yorkville, The Annex, North Midtown
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Harbourfront West, Island airport, South Niagara, King and Spadina, CN Tower, Bathurst Quay, Railway Lands
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Trinity, Little Portugal
Exhibition Place, Brockton, Parkdale Village
The Junction South, High Par

In [293]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
3,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [294]:
toronto_venues.shape

(1711, 7)

#### Now let's check how many venues were returned for each neighbourhood

In [296]:
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,54,54,54,54,54,54
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",85,85,85,85,85,85
Christie,18,18,18,18,18,18
Church and Wellesley,82,82,82,82,82,82
Davisville,39,39,39,39,39,39
Davisville North,8,8,8,8,8,8
"Dovercourt Village, Dufferin",17,17,17,17,17,17


#### Let's find out how many unique categories can be curated from all the returned venues

In [297]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 225 uniques categories.


### Analyze Each Neighborhood

In [298]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Lets examine new dataset size

In [299]:
toronto_onehot.shape

(1711, 225)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [300]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.018519,0.000000,0.000000,0.000000,0.00
1,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,Central Bay Street,0.011905,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.011905,...,0.000000,0.000000,0.000000,0.00000,0.00,0.011905,0.000000,0.000000,0.011905,0.00
3,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.047059,0.000000,0.070588,0.011765,0.00
4,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
5,Church and Wellesley,0.012195,0.012195,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012195,...,0.012195,0.012195,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
6,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.025641,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
7,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
8,"Dovercourt Village, Dufferin",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
9,"Exhibition Place, Brockton, Parkdale Village",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00


#### Let's print each neighborhood along with the top 5 most common venues

In [302]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.06
2              Bakery  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3                Park  0.06
4         Pizza Place  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1      Ice Cream Shop  0.05
2                Café  0.05
3  Italian Restaurant  0.05
4           Juice Bar  0.04


----Chinatown, Grange Park, Kensington Market----
                           venue  freq
0          Vietnamese Restaurant  0.07
1                            Bar  0.07
2                           Café  0.06
3            Dumpling Restaurant  0.05
4  Vegetarian / Vegan Restaurant  0.05


----Christie----
           venue  freq
0  Grocery Store  0.22
1           Café  0.17
2           P

#### Let's put that into a *pandas* dataframe

In [304]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [305]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Café,Steakhouse,Cheese Shop,Seafood Restaurant,Beer Bar,Bistro
1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Park,Pizza Place,Burrito Place,Restaurant,Brewery,Smoke Shop
2,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Burger Joint,Japanese Restaurant,Juice Bar,Sandwich Place,Gym / Fitness Center,Department Store
3,"Chinatown, Grange Park, Kensington Market",Vietnamese Restaurant,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Dumpling Restaurant,Chinese Restaurant,Mexican Restaurant,Donut Shop,Dessert Shop
4,Christie,Grocery Store,Café,Park,Diner,Baby Store,Restaurant,Athletics & Sports,Candy Store,Nightclub,Italian Restaurant


## Cluster the neighborhoods with kmeans

In [307]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [319]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Café,Steakhouse,Cheese Shop,Seafood Restaurant,Beer Bar,Bistro
1,0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Park,Pizza Place,Burrito Place,Restaurant,Brewery,Smoke Shop
2,0,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Burger Joint,Japanese Restaurant,Juice Bar,Sandwich Place,Gym / Fitness Center,Department Store
3,0,"Chinatown, Grange Park, Kensington Market",Vietnamese Restaurant,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Dumpling Restaurant,Chinese Restaurant,Mexican Restaurant,Donut Shop,Dessert Shop
4,0,Christie,Grocery Store,Café,Park,Diner,Baby Store,Restaurant,Athletics & Sports,Candy Store,Nightclub,Italian Restaurant


#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [344]:
toronto_merged = toronto_df

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)

toronto_merged.head() # check the last columns!

/home/felipe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
37,M4E,East Toronto,The Beaches,43.6764,-79.293,0
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6796,-79.3522,0
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.669,-79.3156,0
43,M4M,East Toronto,Studio District,43.6595,-79.3409,0
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888,0


In [380]:
def get_common_venues(data):
    #roww = neighborhoods_venues_sorted[neighborhoods_venues_sorted['Neighborhood'] == row.Neighborhood].iloc[0]
    #indexes = roww.index.to_list()
    #values = roww.to_list()
    #for index,value in zip(indexes,values):
    #    if index.endswith('Venue'):
    #        row[index] = value
    for i in range(data.shape[0]):
           data.iloc[i,6:] = neighborhoods_venues_sorted[neighborhoods_venues_sorted['Neighborhood'] == data.iloc[i].Neighborhood].iloc[0].to_list()[2:]
indexes = neighborhoods_venues_sorted[neighborhoods_venues_sorted['Neighborhood'] == 'The Beaches'].iloc[0].index.to_list()
for index in indexes:
    if index.endswith('Venue'):
        toronto_merged[index] = np.nan
get_common_venues(toronto_merged)
toronto_merged

/home/felipe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/felipe/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.6764,-79.293,0,Pub,Other Great Outdoors,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6796,-79.3522,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Diner,Spa,Bookstore,Brewery
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.669,-79.3156,0,Park,Sandwich Place,Pizza Place,Italian Restaurant,Brewery,Ice Cream Shop,Burger Joint,Burrito Place,Pub,Liquor Store
43,M4M,East Toronto,Studio District,43.6595,-79.3409,0,Café,Coffee Shop,Brewery,Italian Restaurant,Bakery,American Restaurant,Gastropub,Bookstore,Seafood Restaurant,Ice Cream Shop
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888,0,Bus Line,Park,Swim School,Women's Store,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
45,M4P,Central Toronto,Davisville North,43.7128,-79.3902,0,Gym,Breakfast Spot,Department Store,Hotel,Convenience Store,Sandwich Place,Food & Drink Shop,Park,Discount Store,Dance Studio
46,M4R,Central Toronto,North Toronto West,43.7154,-79.4057,0,Coffee Shop,Clothing Store,Yoga Studio,Mexican Restaurant,Diner,Salon / Barbershop,Dessert Shop,Restaurant,Spa,Sporting Goods Shop
47,M4S,Central Toronto,Davisville,43.7043,-79.3888,0,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Sushi Restaurant,Café,Gym,Italian Restaurant,Bowling Alley,Seafood Restaurant
48,M4T,Central Toronto,"Summerhill East, Moore Park",43.6896,-79.3832,0,Gym,Playground,Park,Tennis Court,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Women's Store
49,M4V,Central Toronto,"Rathnelly, Forest Hill SE, Deer Park, Summerhi...",43.6864,-79.4,0,Coffee Shop,Pub,American Restaurant,Restaurant,Light Rail Station,Sports Bar,Liquor Store,Supermarket,Pizza Place,Sushi Restaurant


### Finally, let's visualize the resulting clusters


In [386]:
# create map
map_clusters = folium.Map(location=[43.5000, -79.3], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3 style="color:purple;"> Examine clusters</h3>

### Cluster 1

In [389]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Pub,Other Great Outdoors,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Diner,Spa,Bookstore,Brewery
42,East Toronto,0,Park,Sandwich Place,Pizza Place,Italian Restaurant,Brewery,Ice Cream Shop,Burger Joint,Burrito Place,Pub,Liquor Store
43,East Toronto,0,Café,Coffee Shop,Brewery,Italian Restaurant,Bakery,American Restaurant,Gastropub,Bookstore,Seafood Restaurant,Ice Cream Shop
44,Central Toronto,0,Bus Line,Park,Swim School,Women's Store,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
45,Central Toronto,0,Gym,Breakfast Spot,Department Store,Hotel,Convenience Store,Sandwich Place,Food & Drink Shop,Park,Discount Store,Dance Studio
46,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Mexican Restaurant,Diner,Salon / Barbershop,Dessert Shop,Restaurant,Spa,Sporting Goods Shop
47,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Sushi Restaurant,Café,Gym,Italian Restaurant,Bowling Alley,Seafood Restaurant
48,Central Toronto,0,Gym,Playground,Park,Tennis Court,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Women's Store
49,Central Toronto,0,Coffee Shop,Pub,American Restaurant,Restaurant,Light Rail Station,Sports Bar,Liquor Store,Supermarket,Pizza Place,Sushi Restaurant


### Cluster 2

In [390]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Downtown Toronto,1,Coffee Shop,Bakery,Restaurant,Café,Pizza Place,Pub,Convenience Store,Italian Restaurant,Park,Snack Place
63,Central Toronto,1,Home Service,Garden,Cuban Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant


### Cluster 3

In [391]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Central Toronto,2,Jewelry Store,Park,Trail,Sushi Restaurant,Women's Store,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Discount Store


### Cluster 4

In [393]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,Downtown Toronto,3,Coffee Shop,Café,Steakhouse,Restaurant,Hotel,Bar,Gastropub,Gym,Asian Restaurant,Japanese Restaurant


### Cluster 5

In [394]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,Downtown Toronto,4,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Café,Steakhouse,Cheese Shop,Seafood Restaurant,Beer Bar,Bistro


<h1 style="color:blue;">END OF PART 3</h1>